In [23]:
import numpy as np
import pickle


In [24]:
def random_binary_matrix(m, n, p=0.5):
    A = np.random.binomial(1,p,size=(m,n))
    return A

def dec_to_bin(x, num_bits):
    assert x < 2**num_bits, "number of bits are not enough"
    u = bin(x)[2:].zfill(num_bits)
    u = list(u)
    u = [int(i) for i in u]
    return np.array(u)

def get_sampling_index(x, A, p=0):
    """
    x: sampling index
    A: subsampling matrix
    p: delay
    """
    num_bits = A.shape[0]
    x = dec_to_bin(x, num_bits)
    r = x.dot(A) + p
    return r % 2

def get_random_binary_string(num_bits, p=0.5):
    a = np.random.binomial(1,p,size=num_bits)
    return a

def random_delay_pair(num_bits, target_bit):
    """
    num_bits: number of bits
    location_target: the targeted location (q in equation 26 in https://arxiv.org/pdf/1508.06336.pdf)
    """
    e_q = 2**target_bit
    e_q = dec_to_bin(e_q, num_bits)
    
    random_seed = get_random_binary_string(num_bits)
    
    return random_seed, (random_seed+e_q)%2

def make_delay_pairs(num_pairs, num_bits):
    z = []
    # this is the all zeros for finding the sign
    # actually we do not need this here because we solve
    # a linear system to find the value of the coefficient
    # after the location is found -- however, i am going to
    # keep this here not to have to change the rest of the code
    # that takes delays of this form
    z.append(dec_to_bin(0,num_bits))
    # go over recovering each bit, we need to recover bits 0 to num_bits-1
    for bit_index in range(0, num_bits):
        # we have num_pairs many pairs to do majority decoding
        for pair_idx in range(num_pairs):
            a,b = random_delay_pair(num_bits, bit_index)
            z.append(a)
            z.append(b)
    return z

In [25]:
# the sparsity we target is around K = 2**m
m = 4

# this is the signal length N = 2**n
n = 13

# num delays per single bit of the location index
# (the larger this number the more tolerant to noise we are)
# so one needs to play around with this a bit
d = 5


In [26]:
print('target K around: {}'.format(int((3*2**m)//1.5)))

target K around: 32


In [27]:
# total number of samples from the signal
# from this you can calculate the time necessary
# you can adjust d accordingly to tune the time necessary
# the larger d is better, but then it takes more time too
total_samples = (2**m)*n*(d*2+1)
print('total samples: {}'.format(total_samples))

print('samples/ambient dimension: {}'.format(total_samples/(2**n)))

total samples: 2288
samples/ambient dimension: 0.279296875


We need to run the code below 3 times and save as separate matrices

In [28]:
A = random_binary_matrix(m, n)

sampling_locations_base = []

for i in range(2**A.shape[0]):
    sampling_locations_base.append(get_sampling_index(i,A))
sampling_locations_base = np.array(sampling_locations_base)

delays = make_delay_pairs(d, A.shape[1])
# delays = np.array(delays).shape

In [29]:
# example without the delay
print(sampling_locations_base)

[[0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 0 0 1 1 0 1 1 1 0]
 [0 0 1 1 0 1 0 0 0 0 0 1 1]
 [1 0 0 0 0 1 1 1 0 1 1 0 1]
 [1 1 1 0 0 0 1 1 1 1 1 0 0]
 [0 1 0 1 0 0 0 0 1 0 0 1 0]
 [1 1 0 1 0 1 1 1 1 1 1 1 1]
 [0 1 1 0 0 1 0 0 1 0 0 0 1]
 [1 0 0 0 0 0 0 1 1 0 1 1 0]
 [0 0 1 1 0 0 1 0 1 1 0 0 0]
 [1 0 1 1 0 1 0 1 1 0 1 0 1]
 [0 0 0 0 0 1 1 0 1 1 0 1 1]
 [0 1 1 0 0 0 1 0 0 1 0 1 0]
 [1 1 0 1 0 0 0 1 0 0 1 0 0]
 [0 1 0 1 0 1 1 0 0 1 0 0 1]
 [1 1 1 0 0 1 0 1 0 0 1 1 1]]


In [30]:
all_sampling_locations = []

for current_delay in delays:
    new_sampling_locations = (sampling_locations_base + current_delay) % 2
    all_sampling_locations.append(new_sampling_locations)

This is a list of all matrices of all sampling locations necessary

In [31]:
# example with the delay
print(all_sampling_locations[1])

[[0 1 0 1 1 0 1 0 0 0 0 0 0]
 [1 1 1 0 1 0 0 1 0 1 1 1 0]
 [0 1 1 0 1 1 1 0 0 0 0 1 1]
 [1 1 0 1 1 1 0 1 0 1 1 0 1]
 [1 0 1 1 1 0 0 1 1 1 1 0 0]
 [0 0 0 0 1 0 1 0 1 0 0 1 0]
 [1 0 0 0 1 1 0 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 0 1 0 0 0 1]
 [1 1 0 1 1 0 1 1 1 0 1 1 0]
 [0 1 1 0 1 0 0 0 1 1 0 0 0]
 [1 1 1 0 1 1 1 1 1 0 1 0 1]
 [0 1 0 1 1 1 0 0 1 1 0 1 1]
 [0 0 1 1 1 0 0 0 0 1 0 1 0]
 [1 0 0 0 1 0 1 1 0 0 1 0 0]
 [0 0 0 0 1 1 0 0 0 1 0 0 1]
 [1 0 1 1 1 1 1 1 0 0 1 1 1]]


In [32]:
# change this to change the output file names
run_number = 1


Save the matrix and the delays used to generate the sampling locations because they will be necessary for the algorithm 

In [33]:
pickle.dump(A, open( "N13-m4-d5/sampling-matrix-{}.p".format(run_number), "wb" ) )
pickle.dump(delays, open( "N13-m4-d5/delays-{}.p".format(run_number), "wb" ) )
pickle.dump(all_sampling_locations, open( "N13-m4-d5/sampling-locations-{}.p".format(run_number), "wb" ) )
